<a href="https://colab.research.google.com/github/shihabshahriar16/Apriori-and-FP-Growth/blob/main/Fp_Growth_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!git clone https://github.com/shihabshahriar16/Apriori-and-FP-Growth.git

Cloning into 'Apriori-and-FP-Growth'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


# Resource Followed

https://towardsdatascience.com/fp-growth-frequent-pattern-generation-in-data-mining-with-python-implementation-244e561ab1c3

# FP Tree
FP tree is the core concept of the whole FP Growth algorithm. Briefly speaking, the FP tree is the compressed representation of the itemset database. The tree structure not only reserves the itemset in DB but also keeps track of the association between itemsets


The tree is constructed by taking each itemset and mapping it to a path in the tree one at a time. The whole idea behind this construction is that
More frequently occurring items will have better chances of sharing items


We then mine the tree recursively to get the frequent pattern. Pattern growth, the name of the algorithm, is achieved by concatenating the frequent pattern generated from the conditional FP trees.

In [11]:
from collections import defaultdict, OrderedDict
import csv
from itertools import chain, combinations
from optparse import OptionParser

# Stage 1: FP tree construction

In [1]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

In [2]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

# Stage 2: Mine the main tree and conditional FP trees

**Mine Tree**

Starting from the 1-frequent pattern, we find all the prefix paths and construct the conditional pattern bases with it. With the conditional pattern bases, the tree is constructed using the exact same constructTree function from above. And if the tree can be successfully constructed, we go deeper and start working on that tree. This is where the recursion happens.

In [3]:
def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])] 
    # Start with the lowest frequency
    for item in sortedItemList:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable) 
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup) 
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup,
                       newFreqSet, freqItemList)

**Find Prefix Path**


Get the first occurrence of the item on the tree, which is the head node of the linked list in header table. Then, traverse the tree all the way up to the root to get the prefix path. After that, we go to the next node in linked list and repeat traversing.

In [4]:
def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency

**Ascend FP Tree**


Ascend the tree with self-recursion. Keep appending the items and calling itself until it reaches the root.

In [5]:
def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

# Fp Growth Function

In [6]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

In [7]:
def fpgrowth(itemSetList, minSupRatio, minConf):
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf)
        return freqItems, rules

#Read an already preprocessed dataset

In [39]:
import csv
def getFromFile(fname):
    itemSets = []
    itemSet = set()

    with open(fname, 'r') as file:
        csv_reader = csv.reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            record = set(line)
            for item in record:
                itemSet.add(frozenset([item]))
            itemSets.append(record)
    return itemSet, itemSets

In [40]:
itemset,itemsets = getFromFile('/content/Apriori-and-FP-Growth/Confectionary store transaction dataset.csv')

In [13]:
# Select some number of random transactions
# import random
# itemsets = random.sample(itemsets, 6263)
# len(itemsets)

6263

In [24]:
import time
start_time = time.time()
globalfrq,rules = fpgrowth(itemsets,0.02,0.5)
end_time = time.time()
print(end_time-start_time)

12.503643274307251


In [ ]:
print(globalfrq)

In [ ]:
print(rules)

# Read a dataset that is not yet processed

In [41]:
import pandas
csvFile = pandas.read_csv('/content/Apriori-and-FP-Growth/All-purpose gift store dataset.csv',encoding= 'unicode_escape')
transactions = csvFile.groupby('InvoiceNo')["Description"].apply(list)
csvFile.head()
itemset = set()
itemsets = []
for r in transactions:
  itemsets.append(set(r))
  itemset = itemset.union(set(r))

In [32]:
# Select some number of random transactions
# import random
# itemsets = random.sample(itemsets, 6263)
# len(itemsets)

25900

# Run Algorithm

In [37]:
import time
start_time = time.time()
globalfrq,rules = fpgrowth(itemsets,0.05,0.8)
end_time = time.time()
print(end_time-start_time)

0.33434128761291504


In [ ]:
print(globalfrq)

In [ ]:
print(rules)